In [ ]:
import bittensor as bt
import storage
import redis
import base64
import json
import sys
import os

bt.trace()

db = redis.Redis(host="localhost", port=6379, db=0)
db

In [ ]:
# create a wallet instance that must be registered on the network
wallet = bt.wallet(name="default", hotkey="default")
print(wallet)

# instantiate the metagraph
metagraph = bt.metagraph(
    netuid=8, network="test", sync=True, lite=False
)
print(metagraph)

# Grab the axon you're serving
axon = metagraph.axons[13]
print(axon)

# Create a Dendrite instance to handle client-side communication.
d = bt.dendrite(wallet=wallet)
print(d)

In [ ]:
# Setup CRS for initial round of validation
g, h = storage.utils.setup_CRS(curve="P-256")

# Make a bytes string to test the miner
data = b"this is a random bytestring, long enough to be chunked into segments and reconstructed at the end"

# Encrypt the data
encrypted_data, nonce, tag = storage.utils.encrypt_data(
    data,
    wallet.hotkey.public_key,
)
data_hash = storage.utils.hash_data(encrypted_data)
print("data_hash:", data_hash)

encrypted_data

In [ ]:
# Convert to base64 for compactness
b64_encrypted_data = base64.b64encode(encrypted_data).decode("utf-8")
b64_encrypted_data

In [19]:
# Create the synapse to store it
synapse = storage.protocol.Store(
    encrypted_data=b64_encrypted_data,
    data_hash=data_hash,
    curve="P-256",
    g=storage.utils.ecc_point_to_hex(g),
    h=storage.utils.ecc_point_to_hex(h),
    seed=os.urandom(32).hex(),
)
synapse

Store(encrypted_data='KxxUN+nV0moXH0UKHunqfo9nTRlrYmpkppi94U0CFxjp1plucKf5KoumxMVPP2a9KploQcfo1wq1k37bIgMuTUhyWfGNWPQI7YSeeruPbCC1kaZFa09mKNVorZ0xzOv64A==', curve='P-256', g='33353330383739313133393032383139343032393238353631343534393633353730353132373239373736343233383636353531363538313536343834353632363935323630353639313332372c3837343938343034333437383434343631353530363938393336313538313038313930383134323339323630333834393136353830393034333537343937353037303936363733303435393234', h='35303935383937363238393933363734333331353533363932393739303634343930333735333439363431383831333432383738313237373235383538343939373631363433373639303332302c393137383634343230373236313335353730373733343734343436363731343635373330333934313335343035333933363031373134303833383835393233313732303735323538373534', seed='48819038693a4de1e16a9d892befc3d7b1eea49eaf8ee0a2e19d4d4fd9a7d617', randomness=None, commitment=None, signature=None, commitment_hash=None)

In [25]:
# Send a request to the Axon using the Dendrite, passing in a Synapse 
# instance with the data_hash field. 
responses = await d(
    [axon],
    syn,
    deserialize=False,
)
responses 

2023-11-08 21:00:42.927 |      DEBUG       | dendrite | --> | 4953 B | Store | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 0 | Success


2023-11-08 21:00:42.976 |      DEBUG       | dendrite | <-- | 6150 B | Store | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 200 | Success


[Store(encrypted_data='KxxUN+nV0moXH0UKHunqfo9nTRlrYmpkppi94U0CFxjp1plucKf5KoumxMVPP2a9KploQcfo1wq1k37bIgMuTUhyWfGNWPQI7YSeeruPbCC1kaZFa09mKNVorZ0xzOv64A==', curve='P-256', g='39313938323839343933373633383136353231323034333637383230313632393638383935393039353930313336363339363232323531363138373332323937383538393539333633333938302c3531323831303830393630303137313234393830373532353938343835303935353838373033353339363034303037383437313735313938343731313138333537343139363438303534393433', h='34303430393336303036373632313131373337393831393433303233323138353434303538363938343934333537323934353135383237313031373839313534323339363638373332333936302c38373535343135313931303735373731313530303735373931353732383539383238373931313832323732343138393138353638373932353339303633303735343430353231313638393736', seed='158c71a8bd8730b02176d5ba60d7afe9c000a74ea3fcc936be69a8aa590272e2', randomness=39419905047907825319912365271093218525870954108850638938390052500616957425922, commitment='3734303239323032373934

In [26]:
resp = responses[0]
resp.dict()

{'name': 'Store',
 'timeout': 12.0,
 'total_size': 6150,
 'header_size': 0,
 'dendrite': {'status_code': 200,
  'status_message': 'Success',
  'process_time': 0.05029726028442383,
  'ip': '127.0.0.1',
  'port': 41278,
  'version': 610,
  'nonce': 10385997412510926,
  'uuid': 'bc2feab0-7e78-11ee-8291-3ceceff2faf8',
  'hotkey': '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH',
  'signature': '0x807f526fc77344e343518042199d073f29897e6c469792f36e15d5598a304f3e84af8a81b72d7671f82b449a69556e268109a7f6e32c8a29e2fd8f0f47e09482'},
 'axon': {'status_code': 200,
  'status_message': 'Success',
  'process_time': 0.011322736740112305,
  'ip': '149.137.225.62',
  'port': 8091,
  'version': 610,
  'nonce': 10385997440412281,
  'uuid': 'b30c6a94-7e78-11ee-b8bd-3ceceff2faf8',
  'hotkey': '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH',
  'signature': '0x5cbf2c403439363dd3fd3d3186058724713cd6796886dc1fa7861e652e5f5c20abdb8cbf618b4d45410b6d66f37c0a21aa1116d09e03b7ad0ba1ff6186bd608d'},
 'computed_body

In [28]:
# Store necessary data on the validator side
response_storage = {
    "size": sys.getsizeof(encrypted_data),
    "prev_seed": synapse.seed,
    "commitment_hash": resp.commitment_hash,  # contains the seed
    # these will be private to the validator, not stored in decentralized GUN db
    # just here now for testing purposes
    "encryption_key": wallet.hotkey.public_key.hex(),
    "encryption_nonce": nonce.hex(),
    "encryption_tag": tag.hex(),
}
response_storage

{'size': 130,
 'prev_seed': '48819038693a4de1e16a9d892befc3d7b1eea49eaf8ee0a2e19d4d4fd9a7d617',
 'commitment_hash': '17923109242015911936824406608159296440845322093424120534406591182102475622895',
 'encryption_key': '029635f43d71c3314434ed5898901637fae64ab883d4b3082fb07960e0914227',
 'encryption_nonce': '9111513cf3fc48659815a2ea71080850',
 'encryption_tag': 'd055d63e1ef9d673e36252cec0bc7b4e'}

In [31]:
# Store in the database according to the data hash and the miner hotkey
key = f"{data_hash}.{resp.axon.hotkey}"
print("validator storage key:", key)
db.set(key, json.dumps(response_storage).encode())

validator storage key: 62005888844054250998468105801135578199603310076657368670752933618999799345979.5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH


True

In [32]:
storage.utils.verify_store_with_seed(resp)


Opening: Hashed Value = 17923109242015911936824406608159296440845322093424120534406591182102475622895
Random Value = 39419905047907825319912365271093218525870954108850638938390052500616957425922
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff57e60e830>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff57dd3ece0>


True

In [33]:
# Setup a challenge to the miner with a new CRS and on data we just stored
g, h = storage.utils.setup_CRS(curve="P-256")
challenge = storage.protocol.Challenge(
    challenge_hash=data_hash,
    g=storage.utils.ecc_point_to_hex(g),
    h=storage.utils.ecc_point_to_hex(h),
    curve="P-256",
    seed=os.urandom(32).hex(),
    challenge_index=3,
    chunk_size=storage.utils.get_random_chunksize(sys.getsizeof(data)) // 4, # bytes
)
challenge

Challenge(challenge_hash='62005888844054250998468105801135578199603310076657368670752933618999799345979', challenge_index=3, chunk_size=31, g='33323339323034393234353139373530383038303939353636333536393931323834313530303132373832373634353134303337373036343631383630303539343230313036303431363337382c3337333531383132343231323436303139383436323738343336313534373139373130333738373638363036333330323831333832383336313431303739353235313736393439383136393538', h='38323434343139363538313135303732383337393538383730303935333732303832353138393536383534313131303330373537363434303937353231353737303636333731363736353239372c3632313239353939333638323632393435303832303336343932373931313632323339353538323137363031323437363436383139353335323633353234323133323537383135313633363132', curve='P-256', seed='62b81983d7c20c48fee48bcc48bf6452bb6d01fa76030c395aca12cd9edb78b6', commitment=None, data_chunk=None, randomness=None, merkle_proof=None, merkle_root=None)

In [34]:
# Send the challenge to the miner
responses = d.query(
    [metagraph.axons[13]],
    challenge,
    deserialize=False,
)
responses[0]

2023-11-08 21:02:33.241 |      DEBUG       | dendrite | --> | 5199 B | Challenge | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 0 | Success
2023-11-08 21:02:33.312 |      DEBUG       | dendrite | <-- | 6846 B | Challenge | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 200 | Success


Challenge(challenge_hash='62005888844054250998468105801135578199603310076657368670752933618999799345979', challenge_index=3, chunk_size=31, g='33323339323034393234353139373530383038303939353636333536393931323834313530303132373832373634353134303337373036343631383630303539343230313036303431363337382c3337333531383132343231323436303139383436323738343336313534373139373130333738373638363036333330323831333832383336313431303739353235313736393439383136393538', h='38323434343139363538313135303732383337393538383730303935333732303832353138393536383534313131303330373537363434303937353231353737303636333731363736353239372c3632313239353939333638323632393435303832303336343932373931313632323339353538323137363031323437363436383139353335323633353234323133323537383135313633363132', curve='P-256', seed='62b81983d7c20c48fee48bcc48bf6452bb6d01fa76030c395aca12cd9edb78b6', commitment='33383233323135393130383533333931353434303738343339383834333039383432323431333932343535313333343530363532313738353030393838363132

In [35]:
# Verify the challenge
storage.utils.verify_challenge_with_seed(responses[0])


Opening: Hashed Value = 82925716138693023682401643400271053831383105478961603314525418514502722507415
Random Value = 74322348858501431001741171341432828845003650036256950228289803008964210013931
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff57e5a8ee0>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff57e1ea5f0>


True

In [36]:
# Now retrieve the data we stored and successfully challenged
syn = storage.protocol.Retrieve(
    data_hash=data_hash
)
syn

Retrieve(data_hash='62005888844054250998468105801135578199603310076657368670752933618999799345979', data=None)

In [37]:
# Send a request to the Axon using the Dendrite, passing in a Synapse 
# instance with the data_hash field. 
responses = await d(
    [axon],
    syn,
    deserialize=False,
)
responses 

2023-11-08 21:03:26.445 |      DEBUG       | dendrite | --> | 3397 B | Retrieve | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 0 | Success
2023-11-08 21:03:26.473 |      DEBUG       | dendrite | <-- | 4069 B | Retrieve | 5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH | 149.137.225.62:8091 | 200 | Success


[Retrieve(data_hash='62005888844054250998468105801135578199603310076657368670752933618999799345979', data='KxxUN+nV0moXH0UKHunqfo9nTRlrYmpkppi94U0CFxjp1plucKf5KoumxMVPP2a9KploQcfo1wq1k37bIgMuTUhyWfGNWPQI7YSeeruPbCC1kaZFa09mKNVorZ0xzOv64A==')]

In [38]:
# extract the base64 encoded string of the data
resp = responses[0]
resp.data

'KxxUN+nV0moXH0UKHunqfo9nTRlrYmpkppi94U0CFxjp1plucKf5KoumxMVPP2a9KploQcfo1wq1k37bIgMuTUhyWfGNWPQI7YSeeruPbCC1kaZFa09mKNVorZ0xzOv64A=='

In [39]:
# decode from base64 to bytes
decoded_data = base64.b64decode(resp.data)
decoded_data

b'+\x1cT7\xe9\xd5\xd2j\x17\x1fE\n\x1e\xe9\xea~\x8fgM\x19kbjd\xa6\x98\xbd\xe1M\x02\x17\x18\xe9\xd6\x99np\xa7\xf9*\x8b\xa6\xc4\xc5O?f\xbd*\x99hA\xc7\xe8\xd7\n\xb5\x93~\xdb"\x03.MHrY\xf1\x8dX\xf4\x08\xed\x84\x9ez\xbb\x8fl \xb5\x91\xa6EkOf(\xd5h\xad\x9d1\xcc\xeb\xfa\xe0'

In [41]:
# decrypt back to the original file content bytes
storage.utils.decrypt_aes_gcm(
    decoded_data,
    bytes.fromhex(response_storage['encryption_key']),
    bytes.fromhex(response_storage['encryption_nonce']),
    bytes.fromhex(response_storage['encryption_tag']),
)

b'this is a random bytestring, long enough to be chunked into segments and reconstructed at the end'